<a href="https://colab.research.google.com/github/alekhyam94/DDL-Project/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

import os

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Download training data

In [0]:
!mkdir -p /content/2/frame/train
os.chdir('/content/2/frame/train')
!curl data.yt8m.org/download.py | shard=1,3000 partition=2/frame/train mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0  16300      0 --:--:-- --:--:-- --:--:-- 16300
Starting fresh download in this directory. Please make sure you have >2TB of free disk space!
>> Downloading http://data.yt8m.org/2/download_plans/frame_train.json 102.7%Succesfully downloaded 2_frame_train_download_plan.json 215275 bytes.
Files remaining 1
Downloading: train2161.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainI1.tfrecord 100.0%Succesfully downloaded train2161.tfrecord 270351379 bytes.
Successfully downloaded train2161.tfrecord


All done. No more files to download.


In [0]:
os.chdir('/content/drive/My Drive/youtube-8m-code/code/youtube-8m/')
!python train.py --frame_features --model=FrameLevelLogisticClassifier --feature_names='rgb' --feature_sizes='1024' --train_data_pattern=/content/2/frame/train/train*.tfrecord --train_dir /content/models/frame/sample_model --start_new_model 

python3: can't open file 'train.py': [Errno 2] No such file or directory


# New Section

In [0]:
!pwd

/content/2/frame/train
